# SUM ALL BLOCK
Đoạn code này tính tổng điểm của các khối thi khác nhau (A, B, C, D) cho từng thí sinh trong dữ liệu thi THPT quốc gia. Sau đó, nó xuất ra một file CSV chỉ chứa các thông tin quan trọng bao gồm SBD, tên tỉnh, và tổng điểm của các khối. 



In [ ]:
import pandas as pd

# Load the data
data_df = pd.read_csv('Data/Data.csv')
tinh_df = pd.read_csv('Data/Tinh.csv')

# Merge data based on 'MaTinh'
merged_df = pd.merge(data_df, tinh_df, on='MaTinh', how='left')

# Tính tổng điểm của các khối
merged_df['Tong_Diem_Khoi_A'] = merged_df[['Toan', 'Ly', 'Hoa']].sum(axis=1, skipna=False)

merged_df['Tong_Diem_Khoi_B'] = merged_df[['Toan', 'Sinh', 'Hoa']].sum(axis=1, skipna=False)
merged_df['Tong_Diem_Khoi_C'] = merged_df[['Van', 'Lich su', 'Dia ly']].sum(axis=1, skipna=False)
merged_df['Tong_Diem_Khoi_D'] = merged_df[['Toan', 'Van', 'Ngoai ngu']].sum(axis=1, skipna=False)

# Lọc chỉ các cột cần thiết cho kết quả cuối cùng
total_scores_df = round(merged_df[['SBD', 'Tong_Diem_Khoi_A', 'Tong_Diem_Khoi_B', 'Tong_Diem_Khoi_C', 'Tong_Diem_Khoi_D', 'TenTinh']], 2)

# Xuất kết quả ra file CSV
total_scores_df.to_csv('Total_Scores.csv', index=False)